# Homework Three:  Building a Calculator: Abstract Syntax Trees, Evaluation, and Memory for Variables

## General Homework Policies for CS 320

Please provide answers to these problems by filling in this notebook and submitting it via websubmit by Tuesday 2/13 by midnight. If the lab for this week involved a separate notebook, it is due at the same time. For any problems involving diagrams or drawings, you may try your hand at ASCII art or you may submit an electronic version of your drawings, either using drawing software or by scanning or photographing hand drawings. A message has been posted to Piazza showing you how to automatically include these in your notebook (a nice touch).  Make sure in the latter case that the result is legible; if we can not understand your drawing, it will receive no credit. 

You may submit by Wednesday midnight for a 10% penalty; after that no credit will be given. 

I will drop the lowest homework/lab combination at the end of the term.

For coding problems any <i> reasonable </i> solution, in which you understand the spirit of the exercise and try to learn this week's material through your efforts, will be graded based solely on the test cases. In general, shortcuts to avoid learning the material will be unreasonable. For example, you may not use any Python libraries or functions that make your task dramatically easier (e.g., you may use split(), replace(...), int(...), and float(...), as mentioned below, and any other normal functions, but may NOT use the Python library for regular expressions).  Good Python style, using list comprehensions, Numpy, creating helper functions in addition to what is required, etc., <i>when appropriate,</i> are quite reasonable! Check with me if you have questions about this. 
<p>
<span style="color:red"><b>Lab problems</b> may be worked on collaboratively at the lab and afterwards, with no need to indicate whom you collaborated with. (Of course, if you simply copy, then you will not learn what you need for homeworks and tests, and your grade will reflect your lack of effort.)</span> 

<span style="color:red"><b>Homework problems</b> must be solved individually, and will be subject to the College rules on plagiarism. <b>In particular, you are absolutely forbidden to copy code or answers from another student or from a source other than my slides and materials I provide to you.</b> We will run the code from notebooks through an electronic service that checks for plagiarism, and any violation will be dealt with harshly, up to and including an automatic drop of one letter grade at the end of the term (this policy will be familiar to those who took CS 111 at BU). Significant offences will be referred to the College Academic Misconduct Committee. These policies are necessary to provide a fair environment for you to do your best work with the expectation that the quality and quantity of your efforts will be rewarded appropriately. <b>Please take this as seriously as I do.</b></span> 

<b> Make sure to click Cell -> Run All before submitting notebooks!!</b>

### Lexical analyzer code. DO NOT MODIFY!

In [1]:
# Token definitions

# Token Categories -- slightly different than in HW01 and HW 02

ident = 0          # used in token as (indent, <string> )    
integer = 1        #  (integer, <value> )
floating = 2       #  (floating, <value> )     
plus = 3           #  (plus,)    rest as this one, no attribute necessary
minus = 4          
mult = 5
div = 6            
lparen = 7
rparen = 8
equals = 9
let = 10
inTok = 11
error = 12         #  (error, <string> )     gives spelling of token that caused the lexical error

# Non-terminals are encoded as pairs using these codes:

S = 19             # (S,)  etc. 
E = 13
A = 14
L = 15
T = 16
F = 17
I = 20            # added this in the last step

# special token for end of input

end_of_input = 18       # (end_of_input,) will be pretty-printed as ($,)


# pretty-printing lists of tokens

tokenCats = ['ident','integer','floating','plus','minus','mult','div',
             'lparen','rparen','=','let','in','error','E','A','L','T','F','$','S','I']

tokenSymbols = ['id','int','float','+','-','*','/',
             '(',')','=','let','in','error','E','A','L','T','F','$','S','I']

def tokenToString(t):
    if t == None:
        return str(t)
    elif t[0] < 3:
        return '(' + tokenCats[t[0]] + ',' + str(t[1]) + ')'
    else:
        return '(' + tokenCats[t[0]] + ',)'
        
def tokenListToString(lst):
    res = '[ '
    for t in lst[:-1]:
        res = res + tokenToString(t) + ', '
    res = res + tokenToString(lst[-1]) + ' ]'
    return res


# Code for lexer

# put white space between each token and split into words
def splitTokens(s):
    for t in ['+','-','*','/','(',')',',','=']:
        s = s.replace(t,' ' + t + ' ')
    return s.split()

def isLetter(c):
    return 'a' <= c <= 'z' or 'A' <= c <= 'Z'

def isDigit(c):
    return '0' <= c <= '9' 

def isIdToken(s):
    state = 1
    for ch in s:
        if state == 1:
            if isLetter(ch) or ch == '_':
                state = 2
            else:
                return False
        elif state == 2:
            if isLetter(ch) or isDigit(ch) or ch == '_':
                state = 2
            else:
                return False
    return (state == 2)

                        
def isIntToken(s):
    if s == '0':
        return True
    state = 1
    for ch in s:
        if state == 1:
            if isDigit(ch) and ch != '0':
                state = 2
            else:
                return False
        elif state == 2:
            if isDigit(ch):
                state = 2
            else:
                return False
    return (state == 2)
    
def isFloatToken(s):
    state = 1
    finalStates = [4,6]
    for ch in s:
        if state == 1:
            if isDigit(ch) and ch != '0':
                state = 2
            elif ch == '0':
                state = 3
            elif ch == '.':
                state = 5
            else:
                return False
        elif state == 2:
            if isDigit(ch):
                state = 2
            elif ch == '.':
                state = 4
            else:
                return False
        elif state == 3:
            if ch == '.':
                state = 4
            else:
                return False
        elif state == 4:
            if isDigit(ch):
                state = 4
            else:
                return False
        elif state == 5:
            if isDigit(ch):
                state = 6
            else:
                return False
        elif state == 6:
            if isDigit(ch):
                state = 6
            else:
                return False                
                
    return (state in finalStates)   
    
def isPlusToken(s):
    return s == '+'
    
def isMinusToken(s):
    return s == '-'

def isMultToken(s):
    return s == '*'
    
def isDivToken(s):
    return s == '/'

def isLparenToken(s):
    return s == '('
    
def isRparenToken(s):
    return s == ')'

def isEqualsToken(s):
    return s == '='

def isLetToken(s):
    return s == 'let'

def isInToken(s):
    return s == 'in'

def stringToToken(t):
    if isPlusToken(t):
        return (plus,) 
    elif isMinusToken(t):
        return (minus,) 
    elif isMultToken(t):
        return (mult,) 
    elif isDivToken(t):
        return (div,) 
    elif isLparenToken(t):
        return (lparen,) 
    elif isRparenToken(t):
        return (rparen,)
    elif isEqualsToken(t):
        return (equals,)
    elif isLetToken(t):
        return (let,)
    elif isInToken(t):
        return (inTok,t)
    elif isIdToken(t):
        return (ident,t)
    elif isIntToken(t):
        return (integer,int(t)) 
    elif isFloatToken(t):
        return (floating,float(t)) 

    else:
        return (error,)


def lexer(s):
    return [stringToToken(t) for t in splitTokens(s)]


#   Parser 

Here is the grammar as discussed in lecture for the language of arithmetic expressions over +, -, *, / , with floats and integers, identifiers, and let expressions. We will also provide an assignment operator, with restrictions. 
<p>
We will avoid the pathologies explored in lab by only allowing assignments at the top level, so that in the interactive calculator at the end of the homework, you can bind a value into a variable, but you MAY NOT use an assignment as part of an expression. You can see in the CFG how this has been accomplished at the syntactic level. (This is consistent with Python; see the <a href="http://www.cs.bu.edu/fac/snyder/cs320/AssignmentExpressionsExamples.pdf">PDF</a> that I posted on the web site for examples of the weird and difficult behavior that results when you allow this, as is the case in Java, C, and C++.)


Recall that expressions evaluate to values, but the assignment statement e.g.,  
<pre>
   x = (3+4)         # may only be used as a top-level expression and not as a subexpression!
</pre>
does not yield a value, but has an
effect on the global memory: the first time a particular identifier is given a value, it is entered into the memory (for us, a Python dictionary); after that the value is updated (and the previous value destroyed). 
<p>
The let expression (which yields a value) allows you to make a local binding of a value to a local variable, valid only inside the expression to which is prefaced, e.g., 
<pre>
(let x = 3 in (3 * x) / 2)
</pre>
(The let expression must be enclosed in parentheses in order to be a subexpression of another expression, but in any case this reinforces the <i>scope</i> of the binding, i.e., that there is a local context inside the parentheses where the binding has effect.)
This type of expression will require that a recursive evaluation must carry along a list of variable bindings as, essentially, a stack, so that the value of a local variable will be determined by the closest enclosing definition. Thursday's lecture will consider this in detail. 

### The next two cells contain provided code for the homework; DO NOT MODIFY!


In [2]:
# The CFG. Notice how S defines a top-level expression I, which can either be an assignment A or an expression E.

rules = [((S,),[(I,),(end_of_input,)]),           #  S -> I $
         ((I,),[(A,)]),                           #  I -> A
         ((A,),[(ident,),(equals,),(E,)]),        #  A -> id = E
         ((I,),[(E,)]),                           #  I -> E
         ((F,),[(lparen,),(let,),(A,),(inTok,),(E,),(rparen,)]),      #  F -> (let A in E)
         ((E,),[(E,),(plus,),(T,)]),              #  E -> E + T 
         ((E,),[(E,),(minus,),(T,)]),             #  E -> E - T 
         ((E,),[(T,)]),                           #  E -> T
         ((T,),[(T,),(mult,),(F,)]),              #  T -> T * F
         ((T,),[(T,),(div,),(F,)]),               #  T -> T / F
         ((T,),[(F,)]),                           #  T -> F
         ((F,),[(minus,),(F,)]),                  #  F -> - F
         ((F,),[(lparen,),(E,),(rparen,)] ),      #  F -> ( E )
         ((F,),[(ident,)]),                       #  F -> id
         ((F,),[(integer,)] ),                    #  F -> integer
         ((F,),[(floating,)] ) ]                  #  F -> floating


def toStringRule(r):
    s = tokenCats[r[0][0]] + ' := '
    for t in r[1]:
        s = s + tokenSymbols[t[0]] + ' '
    return s
                          
def pprint_rules(lst): 
    for k in range(len(lst)):
        print(str(k) + ": " + toStringRule(lst[k]))# all we really need to know about the rules (since they are embedded in the DFA) is the

pprint_rules(rules)


0: S := I $ 
1: I := A 
2: A := id = E 
3: I := E 
4: F := ( let A in E ) 
5: E := E + T 
6: E := E - T 
7: E := T 
8: T := T * F 
9: T := T / F 
10: T := F 
11: F := - F 
12: F := ( E ) 
13: F := id 
14: F := int 
15: F := float 


In [3]:
# DFA to drive the shift-reduce parser

# The parser DFA will tell you what to do when the stack is
# in a given configuration.  A positive number gives the
# state transition, a non-positive number k is a reduction by the
# rule -k, i.e., -4 means a reduction by rule 4 (t -> F) with no
# advance in the inputListOfTokens string.  -100 is an error and
# 0 is accept. 

err = -100
accept = 0

table = {1: {0:4, 1:12, 2:11, 3:err, 4:9, 5:err, 6:err, 7:10, 8:err, 9:err, 10:err, 11:err, 12:err, 13:2, 14:3, 15:5, 16:7, 17:8, 18:err, 19:err, 20:17},
2: {0:err, 1:err, 2:err, 3:16, 4:15, 5:err, 6:err, 7:err, 8:err, 9:err, 10:err, 11:err, 12:err, 13:err, 14:err, 15:err, 16:err, 17:err, 18:-3, 19:err, 20:err},
3: {0:err, 1:err, 2:err, 3:err, 4:err, 5:err, 6:err, 7:err, 8:err, 9:err, 10:err, 11:err, 12:err, 13:err, 14:err, 15:err, 16:err, 17:err, 18:-1, 19:err, 20:err},
4: {0:err, 1:err, 2:err, 3:-13, 4:-13, 5:-13, 6:-13, 7:err, 8:-13, 9:13, 10:err, 11:-13, 12:err, 13:err, 14:err, 15:err, 16:err, 17:err, 18:-13, 19:err, 20:err},
5: {0:err, 1:err, 2:err, 3:-4, 4:-4, 5:-4, 6:-4, 7:err, 8:-4, 9:err, 10:err, 11:-4, 12:err, 13:err, 14:err, 15:err, 16:err, 17:err, 18:-4, 19:err, 20:err},
6: {0:4, 1:err, 2:err, 3:err, 4:err, 5:err, 6:err, 7:err, 8:err, 9:err, 10:err, 11:err, 12:err, 13:err, 14:20, 15:err, 16:err, 17:err, 18:err, 19:err, 20:err},
7: {0:err, 1:err, 2:err, 3:-7, 4:-7, 5:18, 6:19, 7:err, 8:-7, 9:err, 10:err, 11:-7, 12:err, 13:err, 14:err, 15:err, 16:err, 17:err, 18:-7, 19:err, 20:err},
8: {0:err, 1:err, 2:err, 3:-10, 4:-10, 5:-10, 6:-10, 7:err, 8:-10, 9:err, 10:err, 11:-10, 12:err, 13:err, 14:err, 15:err, 16:err, 17:err, 18:-10, 19:err, 20:err},
9: {0:4, 1:12, 2:11, 3:err, 4:9, 5:err, 6:err, 7:10, 8:err, 9:err, 10:err, 11:err, 12:err, 13:err, 14:err, 15:err, 16:err, 17:27, 18:err, 19:err, 20:err},
10: {0:4, 1:12, 2:11, 3:err, 4:9, 5:err, 6:err, 7:10, 8:err, 9:err, 10:6, 11:err, 12:err, 13:24, 14:3, 15:5, 16:7, 17:8, 18:err, 19:err, 20:err},
11: {0:err, 1:err, 2:err, 3:-15, 4:-15, 5:-15, 6:-15, 7:err, 8:-15, 9:err, 10:err, 11:-15, 12:err, 13:err, 14:err, 15:err, 16:err, 17:err, 18:-15, 19:err, 20:err},
12: {0:err, 1:err, 2:err, 3:-14, 4:-14, 5:-14, 6:-14, 7:err, 8:-14, 9:err, 10:err, 11:-14, 12:err, 13:err, 14:err, 15:err, 16:err, 17:err, 18:-14, 19:err, 20:err},
13: {0:4, 1:12, 2:11, 3:err, 4:9, 5:err, 6:err, 7:10, 8:err, 9:err, 10:6, 11:err, 12:err, 13:14, 14:3, 15:5, 16:7, 17:8, 18:err, 19:err, 20:err},
14: {0:err, 1:err, 2:err, 3:16, 4:15, 5:err, 6:err, 7:err, 8:err, 9:err, 10:err, 11:-2, 12:err, 13:err, 14:err, 15:err, 16:err, 17:err, 18:-2, 19:err, 20:err},
15: {0:4, 1:12, 2:11, 3:err, 4:9, 5:err, 6:err, 7:10, 8:err, 9:err, 10:err, 11:err, 12:err, 13:err, 14:err, 15:err, 16:28, 17:8, 18:err, 19:err, 20:err},
16: {0:4, 1:12, 2:11, 3:err, 4:9, 5:err, 6:err, 7:10, 8:err, 9:err, 10:err, 11:err, 12:err, 13:err, 14:err, 15:err, 16:29, 17:8, 18:err, 19:err, 20:err},
17: {0:err, 1:err, 2:err, 3:err, 4:err, 5:err, 6:err, 7:err, 8:err, 9:err, 10:err, 11:err, 12:err, 13:err, 14:err, 15:err, 16:29, 17:8, 18:0, 19:err, 20:err},
18: {0:4, 1:12, 2:11, 3:err, 4:9, 5:err, 6:err, 7:10, 8:err, 9:err, 10:err, 11:err, 12:err, 13:err, 14:err, 15:err, 16:err, 17:22, 18:err, 19:err, 20:err},
19: {0:4, 1:12, 2:11, 3:err, 4:9, 5:err, 6:err, 7:10, 8:err, 9:err, 10:err, 11:err, 12:err, 13:err, 14:err, 15:err, 16:err, 17:23, 18:err, 19:err, 20:err},
20: {0:err, 1:err, 2:err, 3:err, 4:err, 5:err, 6:err, 7:err, 8:err, 9:err, 10:err, 11:21, 12:err, 13:err, 14:err, 15:err, 16:err, 17:err, 18:err, 19:err, 20:err},
21: {0:4, 1:12, 2:11, 3:err, 4:9, 5:err, 6:err, 7:10, 8:err, 9:err, 10:6, 11:err, 12:err, 13:26, 14:3, 15:5, 16:7, 17:8, 18:err, 19:err, 20:err},
22: {0:err, 1:err, 2:err, 3:-8, 4:-8, 5:-8, 6:-8, 7:err, 8:-8, 9:err, 10:-err, 11:-8, 12:err, 13:err, 14:err, 15:err, 16:err, 17:err, 18:-8, 19:err, 20:err},
23: {0:err, 1:err, 2:err, 3:-9, 4:-9, 5:-9, 6:-9, 7:err, 8:-9, 9:err, 10:err, 11:-9, 12:err, 13:err, 14:err, 15:err, 16:err, 17:err, 18:-9, 19:err, 20:err},
24: {0:err, 1:err, 2:err, 3:16, 4:15, 5:err, 6:err, 7:err, 8:25, 9:err, 10:err, 11:err, 12:err, 13:err, 14:err, 15:err, 16:err, 17:err, 18:err, 19:err, 20:err},
25: {0:err, 1:err, 2:err, 3:-12, 4:-12, 5:-12, 6:-12, 7:err, 8:-12, 9:err, 10:err, 11:-12, 12:err, 13:err, 14:err, 15:err, 16:err, 17:err, 18:-12, 19:err, 20:err},
26: {0:err, 1:err, 2:err, 3:16, 4:15, 5:err, 6:err, 7:err, 8:5, 9:err, 10:err, 11:err, 12:err, 13:err, 14:err, 15:err, 16:err, 17:err, 18:err, 19:err, 20:err},
27: {0:err, 1:err, 2:err, 3:-11, 4:-11, 5:-11, 6:-11, 7:err, 8:-11, 9:err, 10:err, 11:-11, 12:err, 13:err, 14:err, 15:err, 16:err, 17:err, 18:-11, 19:err, 20:err},
28: {0:err, 1:err, 2:err, 3:-6, 4:-6, 5:18, 6:19, 7:err, 8:-6, 9:err, 10:err, 11:-6, 12:err, 13:err, 14:err, 15:err, 16:err, 17:-6, 18:-6, 19:err, 20:err},
29: {0:err, 1:err, 2:err, 3:-5, 4:-5, 5:18, 6:19, 7:err, 8:-5, 9:err, 10:err, 11:-5, 12:err, 13:err, 14:err, 15:err, 16:err, 17:-5, 18:-5, 19:err, 20:err}}

# this reads a list of tokens (i.e., the parsing stack) and returns the state or action where you end up;
# numbers > 0 represent states, so you must keep shifting on the stack; numbers <= 0 represent
# reduce moves, so that -n = reduce by rule n. 

def action(lst,table):
    state = 1
    #print(str(state) + " ",end="")            # I used these in debugging 
    for token in lst:         
        state = table[state][token[0]] 
        #print(tokenToString(token) + " " + str(state) + " ", end="")
        if state == err:        # error state is implicit so fail as soon as you hit an error
            return err
    #print()
    return state

# test
action([(E,),(plus,),(F,),(rparen,)],table)

-10

## Problem One: Building Abstract Syntax Trees (10 pts)

The code below provides a shift-reduce parser for the augmented grammer above. It is consistent with what you handed in for HW 02, in that it will determine if a string is in the language specified by the augmeted grammar which includes all the arithmetic operators pluy top-level assignments plus the let expression. You can see some examples below where we demonstrate the parser. 
<p>
Your task for this first problem is to modify this parser so that it constructs abstract-syntax trees as discussed in lecture on Tuesday 2/6. You can see from the test cases in the solution PDF what is required. 

In [4]:
# Code for shift-reduce parser

# pretty-printing parser configurations

def pprintParser(parsingStack,inputListOfTokens):
    totalWidth = 80
    smallestGap = 6
    largestStackLength = int(totalWidth*0.6 - smallestGap/2)   # most characters to display
    largestInputLength = totalWidth - largestStackLength - smallestGap
    
    p = '| '
    for symbol in parsingStack:
        p = p + tokenToString(symbol) + ' '
    if len(p) > largestStackLength:
        ind = int(len(p) - largestStackLength + 9)
        p = '| ... ' + p[ind:]
        
    q = ""
    for tok in inputListOfTokens[:-1]:
        q = q + tokenToString(tok) + ' '
    if len(inputListOfTokens) > 0:
        q = q + tokenToString(inputListOfTokens[-1]) 

    if len(q) > largestInputLength:
        ind = int(largestInputLength - 9)
        q = q[:ind] + ' ... ' 
    
    q = q + ' |'
        
    p = p + (' ' * (totalWidth - len(p) - len(q)))
    print(p+q)
    

# parse takes a list of tokens and determines if it is a 
# well-formed arithmetic expression.

def parse(list_of_input_tokens,rules,table,verbose=False):
    
    # add end of input marker
    list_of_input_tokens.append((end_of_input,))
    
    # input stack; use pop(0) to remove front item in list
    input_stack = list_of_input_tokens
    
    # parsing stack; use append to push onto end and pop() to pop from end
    parsing_stack = []
    
    if verbose:
        print('Input Tokens: ' + tokenListToString(input_stack) + '\n')
        pprintParser(parsing_stack,input_stack)
    
    while( len(input_stack) > 0 ):
        
        # Now we will "lookahead" at the next symbol on the input and ask the automaton what to do
        # a positive number means to shift, and a negative number -n means to reduce by rule n
        #print(input_stack)
        n = action(parsing_stack+[input_stack[0]],table)
        #print(parsing_stack)
        if n == accept:   # reduce by start rule, success
            if verbose:
                pprintParser(parsing_stack,input_stack)
            return parsing_stack[0][1]
        elif n == err:     #  problem on stack!
            if verbose:
                pprintParser(parsing_stack,input_stack)
            return None
        elif n > 0:     # shift          
            parsing_stack.append( input_stack.pop(0))           
        else:         # reduce by rule -n            
            LHS = rules[-n][0]
            RHS = rules[-n][1]
            lenRHS = len(RHS)                    # notice that no matching is necessary!
            r = -n;
          
            if(r in [1,3,7,10]):
                right = parsing_stack.pop()
                parsing_stack.append((LHS[0], right[1]))
               
            elif(r in [13,14,15]):
                right1 = parsing_stack.pop()
                newtuple = (right1[0], right1[1])
                parsing_stack.append((LHS[0], newtuple))
               
            elif(r in [5,6,8,9]):
                right2 = parsing_stack.pop()
                right3 = parsing_stack.pop()
                right4 = parsing_stack.pop()
                tuple1 = (right3[0], right4[1], right2[1])
                parsing_stack.append((LHS[0], tuple1))
              
            elif(r in [11]):
                right5 = parsing_stack.pop()
                right6 = parsing_stack.pop() 
                tupleneg = (right6[0], right5[1])
                parsing_stack.append((LHS[0], tupleneg))
               
            elif(r in [12]):
                parsing_stack.pop()
                right7 = parsing_stack.pop()
                parsing_stack.pop()
                tuplee = (right7[0], right7[1])
                parsing_stack.append((LHS[0], right7[1]))
                
            elif(r in [4]):
                right8 = parsing_stack.pop()
                right9 = parsing_stack.pop()
                right10 = parsing_stack.pop()
                right11 = parsing_stack.pop()
                right12 = parsing_stack.pop()
                right13 = parsing_stack.pop()
                tuple2 = (right12[0], right11[1] ,right9[1])
                parsing_stack.append((LHS[0], tuple2))
               
               
            elif(r in [2]):
                nont = parsing_stack.pop()
                right7 = parsing_stack.pop()
                id1 = parsing_stack.pop()
                tupleequal = (right7[0], id1, nont[1])
                parsing_stack.append((LHS[0], tupleequal))   
        
            else:
                return None
        if verbose:
            pprintParser(parsing_stack,input_stack)
            
    return None     # this will never be executed



In [5]:
# # # demonstrations of the parser -- you may remove these if you like
# s = 'x + 3 / (flag_input - -3)'
# print('Input String: ' + s + '\n')
# print(parse(lexer(s),rules,table))

# s = '(((6)))'
# print('\nInput String: ' + s + '\n')
# print(parse(lexer(s),rules,table))

# s = 'x = x + 1'
# print('\nInput String: ' + s + '\n')
# print(parse(lexer(s),rules,table))

# s = '(let x = 7 in (let y = 8 in x * y))'
# print('\nInput String: ' + s + '\n')
# print(parse(lexer(s),rules,table))

# s = 'x + (let y = (let z = 4 in z) * 2 in (let z = (let b = 4 in b) in (let c = (let x = 4 in b) in x))) + 2'
# print('\nInput String: ' + s + '\n')
# print(parse(lexer(s),rules,table))

# s = 'x + (y = 5)'
# print('\nIllegal Input String: ' + s + '\n')
# print(parse(lexer(s),rules,table))

# s = 'let x = 4 in x*2'
# print('\nIllegal Input String: ' + s + '\n')
# print(parse(lexer(s),rules,table))

In [6]:
# Pretty-printing code for testing your solution

def ASTToString(t):
    if t[0] in [ident,integer,floating]:
        return tokenToString(t)
    elif len(t) == 2:
        return '(' + tokenCats[t[0]] + ',' + ASTToString(t[1]) + ')'
    elif len(t) == 3:
        return '(' + tokenCats[t[0]] + ',' + ASTToString(t[1]) + ',' + ASTToString(t[2]) + ')'
    #else:
    print("** " + str(len(t)))
    print("** " + str(t))
    return "ERROR!"
   
def pprintAST(t):
    pprintASTHelper(t,'')

def pprintASTHelper(t,indent):
    if (t[0] in [ident,integer,floating]):
        print(indent + '(' + tokenCats[t[0]] + ',' + str(t[1]) + ')')
    else:
        print(indent + tokenCats[t[0]])
        for k in range(1,len(t)):
            pprintASTHelper(t[k],indent + '\t')
            
def testParserAST(s,verbose=False):
    t = parse(lexer(s),rules,table,verbose)
    if t == None:
        print("Error: No AST generated!")
    else:
        print("\nAbstract Syntax Tree:\n")
        print(ASTToString(t)) 
        print()
        pprintAST(t)

In [7]:
# Test 1
s = '(5+4) * (3.4 / -x) - 4'
testParserAST(s) 


Abstract Syntax Tree:

(minus,(mult,(plus,(integer,5),(integer,4)),(div,(floating,3.4),(minus,(ident,x)))),(integer,4))

minus
	mult
		plus
			(integer,5)
			(integer,4)
		div
			(floating,3.4)
			minus
				(ident,x)
	(integer,4)


In [8]:
# Test 2
s = 'x = (x + 4 * (3))'
testParserAST(s) 


Abstract Syntax Tree:

(=,(ident,x),(plus,(ident,x),(mult,(integer,4),(integer,3))))

=
	(ident,x)
	plus
		(ident,x)
		mult
			(integer,4)
			(integer,3)


In [9]:
# Test 3
s = '(let x = 7 * 2 in x / -2)'
testParserAST(s) 


Abstract Syntax Tree:

(let,(=,(ident,x),(mult,(integer,7),(integer,2))),(div,(ident,x),(minus,(integer,2))))

let
	=
		(ident,x)
		mult
			(integer,7)
			(integer,2)
	div
		(ident,x)
		minus
			(integer,2)


In [10]:
# Test 4
s = '(let x = 3 * (-(4.5)) + 8 + (2 + 4.2) in (7 + x))'
testParserAST(s) 


Abstract Syntax Tree:

(let,(=,(ident,x),(plus,(plus,(mult,(integer,3),(minus,(floating,4.5))),(integer,8)),(plus,(integer,2),(floating,4.2)))),(plus,(integer,7),(ident,x)))

let
	=
		(ident,x)
		plus
			plus
				mult
					(integer,3)
					minus
						(floating,4.5)
				(integer,8)
			plus
				(integer,2)
				(floating,4.2)
	plus
		(integer,7)
		(ident,x)


In [11]:
# Test 5
s = 'x = (let x = 7 in (let y = 8 in x * y))'
testParserAST(s) 


Abstract Syntax Tree:

(=,(ident,x),(let,(=,(ident,x),(integer,7)),(let,(=,(ident,y),(integer,8)),(mult,(ident,x),(ident,y)))))

=
	(ident,x)
	let
		=
			(ident,x)
			(integer,7)
		let
			=
				(ident,y)
				(integer,8)
			mult
				(ident,x)
				(ident,y)


## Problem Two: Evaluator for Abstract Syntax Trees (10 pts)

For this problem you must write an evaluator for the ASTs that you generated in the previous question.
I suggest doing this in several stages, and, as usual, checking your solution at each step and making sure it is "bullet-proof" before proceeding. 

  - Step One: Write a simple recursive evaluator for arithmetic expressions without variables or let expressions
  - Step Two: Add the ability to add variable bindings to the global memory: when evaluating an assignment, you  will evaluate the expression on the RHS and add that binding to the global memory; when encountering a variable in an expression, you must look up the value in the memory. Assume that there are no unbound variables.
  - Step Three: Add the ability to raise an unbound variable exception (some details are provided below)
  - Step Four: Add the ability to evaluate let expressions by adding an env parameter to the eval: when you evaluate a let expression, recursively call eval on the expression with the local variable binding added to the environment. 
  
Test your program on the examples shown, which progress through these various steps. 

Note that this problem does not use assignments; we will add them in the final problem. 


In [12]:
## Evaluator for ASTs:      

# eval(t,env,memory) takes an abstract syntax tree, an environment, and a global memory and evaluates
# the term in the context of the bindings in env, which is a stack, so
# the first occurrence of the binding is the one that is used; if a variable
# is not found in the env, it is looked for in the global memory; if not there
# an unbound variable exception is raised

# global memory is a dictionary of bindings, e.g., { 'x': 4.5, 'y':-1 }

# bindings are in the form of a tuple (str,value), e.g., ('x',4.5) so
# environment is a list of bindings, e.g., [ ('x',4.5), ('y',-1), ('x',4) ]

# for efficiency, env stack grows from left to right, so push binding on env for
# the recursive call by using env + [binding]

# Use the following for errors with unbound variables

class UnboundVariableException(Exception):
    def __init__(self, name):
        self.name = name
        
# To raise an exception involving a variable (ident, s), use   raise UnboundVariableException(s) 
# Don't worry about catching exceptions, we will do that in the next problem


# look up a variable id (the string representation, not the whole token) in the
# environment from end of list to beginning; if there return the value; if not there, look in memory, if
# not there, raise an exception:   raise UnboundVariableException(name) 



def lookup(name,env,memory):
    for i in env[::-1] :
        if(name == i[0]):
            return i[1]
    if name in memory:
        return memory[name]        
    raise UnboundVariableException(name)
def eval(t,env,memory):
 
    if t[0] == integer:
        return t[1]
    elif t[0] == ident:
        return lookup(t[1], env, memory)

    elif t[0] == mult:
        return eval(t[1], env, memory) * eval(t[2], env, memory)
    elif t[0] == div:
        return eval(t[1], env, memory) / eval(t[2], env, memory)
    elif t[0] == plus:
        return eval(t[1], env, memory) + eval(t[2], env, memory)
    elif t[0] == minus:
        return eval(t[1], env, memory) - eval(t[2], env, memory)
    elif t[0] == let:
        result = eval(t[1][2], env, memory)
        env = env + [(t[1][1][1], result)]
        return eval(t[2], env, memory)
    elif t[0] == equals:
        result = eval(t[2], env, memory)
        memory[t[1][1]] = result
        return result
     



In [13]:
# test 1
s = '3 * 4'
t = parse(lexer(s),rules,table)
print("Should be 12:")
print(eval(t,[],{})) 

Should be 12:
12


In [14]:
# test 2
s = 'x * x'
t = parse(lexer(s),rules,table)
print("Should be 64:")
print(eval(t,[('x',4), ('x',8)],{ 'x' : 1})) 

Should be 64:
64


In [15]:
# test 3
s = 'x * y / z'
t = parse(lexer(s),rules,table)
print("Should be 6.0:")
print(eval(t,[('y',4), ('x',3)],{ 'z' : 2})) 

Should be 6.0:
6.0


In [16]:
# test 4
s = '(let x = 4 in x + y * z)'
t = parse(lexer(s),rules,table)
print("Should be 12:")
print(eval(t,[('y',4), ('x',3)],{ 'z' : 2 })) 

Should be 12:
12


In [17]:
# test 5
s = '(let x = x+1 in (let x = x+2 in (let x = x+3 in x)))'
t = parse(lexer(s),rules,table)
print("Should be 8:")
print(eval(t,[],{ 'x' : 2 })) 

Should be 8:
8


In [ ]:
# test 6
s = 'z * (let x = (let y = 4 in y - x) in (let y = x in z + x + y))'
t = parse(lexer(s),rules,table)
print("Should be 16:")
print(eval(t,[],{ 'z' : 2, 'x' : 1 })) 

Should be 16:
16


## Problem Three: Building a Simple Interactive Calculator (similar to Python)  [5 pts]

To see the whole motivation for all these code, let's finish by creating an interactive environment which can create bindings in a global environment using assignments and evaluate the expressions in our language. Use the solution PDF to see what the expected behavior should be. 

In [ ]:
# Basic Interpreter

# use the following as a framework for imitating what I provide in the solution pdf
# to catch an exception, use something like this:
#            try:
#                print("Out["+str(lineNum)+"]: " + str(eval(t,[])))
#            except UnboundVariableException as n:
#               print("Unbound Variable: " + str(n))

lineNum = 1

global_memory = { }

s = input("\nMyPython 1.0\nInput equation or type \"quit\" to quit:\n\nIn ["+str(lineNum)+"]: ")

while True:
    
    # replace this with code to simulate the behavior expected
    if(s == 'quit'):
        print("\nBye!")
        break
    elif(s == 'global'):
        print(global_memory)
    
    else:
        t = parse(lexer(s),rules,table)
        if(t == None):
            print("Error, Try Again!")
        else:
            try:

                print("Out["+str(lineNum)+"]: " + str(eval(t,[],global_memory)))
            except UnboundVariableException as n:
                  print("Unbound Variable: " + str(n))

    lineNum += 1
    s = input("\nIn ["+str(lineNum)+"]: ")

  